## .envファイルの作成

次のセルを実行して、環境変数を格納するための`.env`ファイルを作成してください。セルの実行は、セルの左上の再生マークをクリックすることでできます。

初回実行時には、カーネルの選択を求められる場合があります。その際には、「Python 環境…」をクリックして「★Python 3.11.9…」を選択してください。

In [ ]:
%%bash

cd ..

# .envファイルを作成する
cat <<EOF > .env
SEARCH_SERVICE_ENDPOINT="your_search_service_endpoint"
SEARCH_API_KEY="your_search_api_key"
AOAI_ENDPOINT="your_aoai_endpoint"
AOAI_API_VERSION=2024-06-01
AOAI_API_KEY="your_aoai_api_key"
EOF

echo ".envファイルが作成されました。"

## 環境変数の設定
Azure Portalで各環境変数を確認し、先ほど作成した`.env`ファイルに格納してください。

1. `SEARCH_SERVICE_ENDPOINT,SEARCH_API_KEY`の設定
    - `SEARCH_SERVICE_ENDPOINT`は、AI Searchリソースの「概要」タブから確認できます。
    ![Image 1](../assets/env1.png)

    - `SEARCH_API_KEY`は、AI Searchリソースの「設定」>「キー」タブから確認できます。
    ![Image 2](../assets/env2.png)

2. `AOAI_ENDPOINT,AOAI_API_KEY`の設定
    - `AOAI_ENDPOINT,AOAI_API_KEY`は、OpenAI Servicesリソースの「リソース管理」>「キーとエンドポイント」タブから確認できます。
    ![Image 3](../assets/env3.png)

#### **※ここまで完了されたら、以降順番にセルを実行してRAGの実装を体験してみてください。**

## 必要なライブラリのインポート
以下のセルで必要なライブラリをインポートします

In [1]:
import os
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes.models import *
from langchain.text_splitter import RecursiveCharacterTextSplitter
from azure.search.documents.models import VectorizedQuery
from PyPDF2 import PdfReader
from openai import AzureOpenAI
from dotenv import load_dotenv

## 環境変数の読み込み
envファイルから環境変数を読み込みます。

In [2]:
# 環境変数からAzure AI Search、Azure OpenAIのエンドポイント等を取得する
load_dotenv()
search_endpoint = os.environ["SEARCH_SERVICE_ENDPOINT"]
search_api_key = os.environ["SEARCH_API_KEY"]
aoai_endpoint = os.environ["AOAI_ENDPOINT"]
aoai_api_version = os.environ["AOAI_API_VERSION"]
aoai_api_key = os.environ["AOAI_API_KEY"]


## Azure AI Searchのインデックスを作成する

1. **クライアントの生成**: `SearchIndexClient`を使用して、Azure AI Searchのインデックスクライアントを作成します。
2. **インデックスの確認**: インデックスがすでに存在する場合、再作成を避けるために何もしません。
3. **フィールドの定義**: インデックスに含まれるフィールドを定義します。ここでは、ドキュメントID、コンテンツ、コンテンツベクトルのフィールドを設定します。
4. **ベクトル検索の設定**: ベクトル検索の設定を行います。
5. **インデックスの作成**: 定義した設定を用いてインデックスを作成します。

以下のコードを実行して、インデックスを作成します。実行後にAI Searchのリソースの「検索管理」>「インデックス」タグでdocsというインデックスが作成されていることを確認してください。



In [3]:
def create_index():
    """
    Azure AI Searchのインデックスを作成する
    """
    client = SearchIndexClient(endpoint= search_endpoint, credential=AzureKeyCredential(search_api_key))
    name = "docs"

    # すでにインデックスが作成済みである場合には何もしない
    if 'docs' in client.list_index_names():
        print("すでにインデックスが作成済みです")
        return

    # インデックスのフィールドを定義する
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="content", type="Edm.String", analyzer_name="ja.microsoft"),
        SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile")
    ]

    # ベクトル検索のための定義を行う
    vector_search = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(
                name="myHnsw"
            )
        ],
        profiles=[
            VectorSearchProfile(
                name="myHnswProfile",
                algorithm_configuration_name="myHnsw",
            )
        ]
    )

    # インデックスを作成する
    index = SearchIndex(name=name, fields=fields, vector_search=vector_search)
    client.create_index(index)
    print("インデックスが作成されました")

# インデックスを作成する
create_index()


インデックスが作成されました


## ドキュメントをAzure AI Searchにインデクシングする

1. **PDFからテキストを抽出**: `PdfReader`を使用してPDFファイルからテキストを抽出します。ここでは、ドキュメントととして厚生労働省が提供する[モデル就業規則](https://www.mhlw.go.jp/content/001018385.pdf)を使用します。
※**リポジトリをForkした際にリポジトリ名を変更された方は、`filepath`の`aoai-rag-handson`部分をご自身のリポジトリ名に変更してください。**
2. **テキストのチャンク化**: テキストを指定したサイズでチャンクに分割します。これにより、大きなテキストを小さな部分に分けて処理しやすくします。
3. **インデクシング**: チャンク化されたテキストをAzure AI Searchにインデックスします。ここでは、Azure OpenAIを使用してテキストのベクトルを生成し、それを含むドキュメントをAzure AI Searchにアップロードします。

以下のコードを順次実行して、ドキュメントをインデックスします。


In [4]:
def extract_text_from_docs(filepath):
    """
    PDFからテキストを抽出する
    """
    print(f"{filepath}内のテキストを抽出中...")
    with open(filepath, "rb") as f:
        reader = PdfReader(f)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    print("テキストの抽出が完了しました")
    return text

# 対象ファイルパスのファイルを読み込んで、Azure AI Searchにインデックスする
filepath = "/workspaces/aoai-rag-handson/data/001018385.pdf"  # 対象ファイルパス
content = extract_text_from_docs(filepath)

/workspaces/aoai-rag-handson/data/001018385.pdf内のテキストを抽出中...
テキストの抽出が完了しました


In [5]:
def create_chunk(content: str, separator: str, chunk_size: int = 512, overlap: int = 0):
    """
    テキストを指定したサイズで分割する
    """
    splitter = RecursiveCharacterTextSplitter(chunk_overlap=overlap, chunk_size=chunk_size, separators=separator)
    chunks = splitter.split_text(content)
    return chunks

# テキストを指定したサイズで分割する
chunksize = 1000  # チャンクサイズ
overlap = 200  # オーバーラップサイズ
separator = ["\n\n", "\n", "。", "、", " ", ""]  # 区切り文字
chunks = create_chunk(content, separator, chunksize, overlap)
print("チャンク化完了")

チャンク化完了


In [6]:
def index_docs(chunks: list):
    """
    ドキュメントをAzure AI Searchにインデックスする
    """
    # Azure AI SearchのAPIに接続するためのクライアントを生成する
    searchClient = SearchClient(
        endpoint=search_endpoint,
        index_name="docs",
        credential=AzureKeyCredential(search_api_key)
    )

    # Azure OpenAIのAPIに接続するためのクライアントを生成する
    openAIClient = AzureOpenAI(
        api_key=aoai_api_key,
        api_version=aoai_api_version,
        azure_endpoint=aoai_endpoint
    )

    # チャンク化されたテキストとそのテキストのベクトルをAzure AI Searchにアップロードする
    for i, chunk in enumerate(chunks):
        print(f"{i+1}個目のチャンクを処理中...")
        response = openAIClient.embeddings.create(
            input=chunk,
            model="text-embedding-3-small-deploy"
        )

        # チャンク化されたテキストとそのテキストのベクトルをAzure AI Searchにアップロードする
        document = {"id": str(i), "content": chunk, "contentVector": response.data[0].embedding}
        searchClient.upload_documents([document])
    print("インデクシング完了")

# テキストをAzure AI Searchにインデックスする
index_docs(chunks)

1個目のチャンクを処理中...
2個目のチャンクを処理中...
3個目のチャンクを処理中...
4個目のチャンクを処理中...
5個目のチャンクを処理中...
6個目のチャンクを処理中...
7個目のチャンクを処理中...
8個目のチャンクを処理中...
9個目のチャンクを処理中...
10個目のチャンクを処理中...
11個目のチャンクを処理中...
12個目のチャンクを処理中...
13個目のチャンクを処理中...
14個目のチャンクを処理中...
15個目のチャンクを処理中...
16個目のチャンクを処理中...
17個目のチャンクを処理中...
18個目のチャンクを処理中...
19個目のチャンクを処理中...
20個目のチャンクを処理中...
21個目のチャンクを処理中...
22個目のチャンクを処理中...
23個目のチャンクを処理中...
24個目のチャンクを処理中...
25個目のチャンクを処理中...
26個目のチャンクを処理中...
27個目のチャンクを処理中...
28個目のチャンクを処理中...
29個目のチャンクを処理中...
30個目のチャンクを処理中...
31個目のチャンクを処理中...
32個目のチャンクを処理中...
33個目のチャンクを処理中...
34個目のチャンクを処理中...
35個目のチャンクを処理中...
36個目のチャンクを処理中...
37個目のチャンクを処理中...
38個目のチャンクを処理中...
39個目のチャンクを処理中...
40個目のチャンクを処理中...
41個目のチャンクを処理中...
42個目のチャンクを処理中...
43個目のチャンクを処理中...
44個目のチャンクを処理中...
45個目のチャンクを処理中...
46個目のチャンクを処理中...
47個目のチャンクを処理中...
48個目のチャンクを処理中...
49個目のチャンクを処理中...
50個目のチャンクを処理中...
51個目のチャンクを処理中...
52個目のチャンクを処理中...
53個目のチャンクを処理中...
54個目のチャンクを処理中...
55個目のチャンクを処理中...
56個目のチャンクを処理中...
57個目のチャンクを処理中...
58個目のチャンクを処理中...
59個目のチャンクを処理中...
60個目のチ

## プロンプトのベクトル化

1. **OpenAIクライアントの作成**: OpenAI APIに接続するためのクライアントを作成します。
2. **プロンプトのベクトル化**: 指定したプロンプトをOpenAIモデルを使用してベクトル化します。このベクトルは、後の検索クエリとして使用されます。`prompt`については、任意のものに変更していただいても構いません。

以下のコードを実行して、プロンプトをベクトル化します。


In [7]:
# OpenAIクライアントの作成
openAIClient = AzureOpenAI(
    api_key=aoai_api_key,
    api_version=aoai_api_version,
    azure_endpoint=aoai_endpoint
)

# プロンプトをベクトル化する関数
def generate_embeddings(prompt, model="text-embedding-3-small-deploy"): # model = "deployment_name"
    response = openAIClient.embeddings.create(input=prompt, model=model).data[0].embedding
    return response

# プロンプトをベクトル化
prompt = "就業時間に関してどのような規定があるのか重要度順に3つ教えてください"
vectorized_prompt = generate_embeddings(prompt)
print(f"ベクトル化したプロンプト{vectorized_prompt[:50]}")

ベクトル化したプロンプト[-0.007567174732685089, 0.01277646142989397, 0.07194299250841141, -0.009398650377988815, 0.03917383775115013, 0.012919032014906406, 0.012765495106577873, -0.005538294557482004, 0.021451294422149658, -0.01731676608324051, 0.0035368315875530243, 0.050316229462623596, -0.009097060188651085, 0.009820876643061638, -0.010478892363607883, -0.03226467967033386, -0.019872058182954788, -0.03000549226999283, 0.021506130695343018, 0.04116981849074364, 0.011953942477703094, 0.024368496611714363, -0.024851040914654732, 0.03366844356060028, 0.005922136828303337, -0.013522212393581867, 0.035159945487976074, -0.002153628971427679, 0.024280760437250137, -0.05062330141663551, -0.0041701714508235455, -0.04320966452360153, -0.005724732298403978, -0.008680317550897598, -0.005409433040767908, 0.03362457826733589, 0.014662771485745907, 0.026627682149410248, -0.00857613142579794, -0.0404021330177784, -0.014070558361709118, -0.020617809146642685, 0.040116991847753525, -0.07957597076892853, -0.043275

## ベクトル検索

1. **SearchClientの作成**: Azure AI Searchに接続するためのクライアントを作成します。
2. **ベクトル検索の準備**: ベクトルクエリを作成し、指定されたフィールドで上位の結果を取得します。
3. **検索実行と結果取得**: 検索を実行し、最初の検索結果を取得します。

以下のコードを実行して、ベクトル検索を行います。


In [8]:
# Azure AI SearchのAPIに接続するためのクライアントを生成する
searchClient = SearchClient(
    endpoint=search_endpoint,
    index_name="docs",
    credential=AzureKeyCredential(search_api_key)
)

# ベクトルクエリの作成
vector_query = VectorizedQuery(
    vector=vectorized_prompt,
    k_nearest_neighbors=3,  # 上位3件の結果を取得します
    fields="contentVector"  # ベクトル検索を行うフィールドを指定します
)

# ベクトル検索の実行
results = searchClient.search(
    search_text='',  # ベクトル検索のみ行うためテキストクエリは空
    vector_queries=[vector_query],
    select=['id', 'content'],
)

# 最初の検索結果を取得
first_result = next(results, None)
if first_result:
    print(first_result["content"])
else:
    print("検索結果が見つかりませんでした")


す。）は、１週４４時間まで働かせることが認められています（労基法第４０条、労基則
第２５条の２）。 
また、労基法第３２条第２項において、１日の労働時間の上限は８時間と定められて
います。 
３ 休憩時間については、１日の労働時間が６時間を超える場合には少なくとも４５分、
８時間を超える場合には少なくとも１時間の休憩時間を与えなければなりません（労基
法第３４条）。 
４ 休日については、毎週少なくとも１回又は４週間を通じ４日以上与えなければなりま
せん（労基法第３５条）。 
５ 上記２から４までの労基法の規定に適合する労働条件とするためには、①週休２日制
とする、②週休１日制で１日の所定労働時間を短く設定する、③変形労働時間制（１か
月単位、１年単位等）を導入する等の方法がありますので、それぞれの事業場の実情に
応じて、下記の規程例を参考に就業規則を作成してください。 
 
 
 
［例１］ 完全週休２日制を採用する場合の規程例 
 
１日の労働時間を８時間とし、完全週休２日制を採用する場合の規程例です。 
 
(労働時間及び休憩時間)  
第１９条  労働時間は、１週間については４０時間、１日については８時間とする。 
２ 始業・終業の時刻及び休憩時間は、次のとおりとする。ただし、業務の都合その他
やむを得ない事情により、これらを繰り上げ、又は繰り下げることがある。この場合、   
前日までに労働者に通知する。 
 
 - 23 - 
 ① 一般勤務 
始業・終業時刻 休憩時間 
始業  午前  時  分 
  時  分から  時  分まで 
終業  午後  時  分 
 
② 交替勤務 
 （イ）１番（日勤） 
始業・終業時刻 休憩時間 
始業  午前  時  分 
  時  分から  時  分まで 
終業  午後  時  分 
 
（ロ）２番（準夜勤） 
始業・終業時刻 休憩時間 
始業  午前  時  分 
  時  分から  時  分まで 
終業  午後  時  分 
 
（ハ）３番（夜勤） 
始業・終業時刻 休憩時間 
始業  午前  時  分 
  時  分から  時  分まで 
終業  午後  時  分 
 
３ 交替勤務における各労働者の勤務は、別に定めるシフト表により、前月の   日
までに各労働者に通知する。


## Azure OpenAIに回答生成依頼

1. **システムメッセージの定義**: GPT-4o-miniに対するシステムメッセージを定義し、AIのキャラクターや回答スタイルを設定します。
2. **ユーザーメッセージの作成**: 検索クエリと検索結果を含むユーザーメッセージを作成します。
3. **回答生成の依頼**: Azure OpenAIに対して、ユーザーメッセージに基づいた回答を生成するよう依頼します。
4. **回答の表示**: 生成された回答を表示します。

以下のコードを実行して、ベクトル検索の結果に基づいた回答を生成します。


In [9]:
# Azure OpenAIクライアントの作成
openAIClient = AzureOpenAI(
    api_key=aoai_api_key,
    api_version=aoai_api_version,
    azure_endpoint=aoai_endpoint
)

# システムメッセージの定義
system_message_chat_conversation = """
あなたはユーザーの質問に回答するチャットボットです。
回答については、「Sources:」以下に記載されている内容に基づいて回答してください。
回答は簡潔にしてください。
「Sources:」に記載されている情報以外の回答はしないでください。
また、ユーザーの質問に対して、Sources:以下に記載されている内容に基づいて適切な回答ができない場合は、「すみません。わかりません。」と回答してください。
回答の中に情報源の提示は含めないでください。例えば、回答の中に「Sources:」という形で情報源を示すことはしないでください。
"""

# ユーザーメッセージの作成
user_message = """
{query}

Sources:
{source}
""".format(query=prompt, source=first_result["content"])

# メッセージリストの作成
messages_for_vector_answer = [
    {"role": "system", "content": system_message_chat_conversation},
    {"role": "user", "content": user_message}
]

# Azure OpenAI Serviceに回答生成を依頼
response = openAIClient.chat.completions.create(
    model="gpt-4o-mini-deploy",
    messages=messages_for_vector_answer
)

# 生成された回答を表示
response_text = response.choices[0].message.content
print(response_text)


1. 労働時間について、週間４４時間までの働かせることが認められ、１日の労働時間の上限は８時間と法律で定められている。
2. 休憩時間について、1日の労働時間が６時間を超える場合には少なくとも４５分、８時間を超える場合には少なくとも１時間の休憩時間を与えなければならない。 
3. 休日については、毎週少なくとも１回又は４週間を通じ４日以上与えなければならない。


## ハンズオンは以上になります
終了された方は、不要なコストの発生を避けるためにお手数ですがAzureリソースの削除・Codespacesの停止をお願い申し上げます。
また、早めに終了された方のために、[追加コンテンツ](#extra1)もご用意しておりますので、リソース削除前にご利用ください。

<a id="delete"></a>
## Azureリソースの削除方法
不要なコストの発生を避けるために以下の手順でリソースを削除しましょう。

- [Azure Portal](https://portal.azure.com/)にアクセスします。画面上部の検索ボックスに「リソースグループ」と入力し、「リソースグループ」というサービスをクリックしてください。
![Image 5](../assets/delete1.png)

- リソースグループの一覧が表示されるので、このセミナーのために作成したリソースグループをクリックしてください。
- リソースグループの「概要」タブの上部にある「リソースグループの削除」をクリックしてください。
![Image 6](../assets/delete2.png)

- 展開されたページ下部の入力欄にリソースグループ名を入力して、「削除」ボタンをクリックしてください。リソースグループに依存するAzure OpenAI ServiceやAzure AI Searchのリソースも同時に削除されます。
![Image 7](../assets/delete3.png)

## Codespacesの停止方法
ハンズオン終了時には、課金が発生しないように忘れずにCodespacesを停止してください。Forkしたリポジトリに移動して、AcitiveなCodespaces横の三点リーダーを展開して、「Stop codespaces」をクリックしてください。

![Image 4](../assets/stop.png)

<a id="extra1"></a>
## 追加コンテンツ
2つの追加コンテンツを用意しました。お時間ある方は試してみてください。

- もっといろんなことを聞いてみよう
- [チャンクサイズの変化による回答内容の変化](#extra2)

### もっといろんなことを聞いてみよう

以下のセルを実行するとプロンプトの入力が求められるので、[モデル就業規則](https://www.mhlw.go.jp/content/001018385.pdf)に関して色々聞いてみましょう。全く関係ないことを聞いた場合にはどんな回答が得られるか試してみましょう。また、モデルの振る舞いを制御したり、性格付けするための`system_message_chat_conversation`を自由に書き換えて、生成される回答がどのように変わるか体験してみてください。

In [10]:
# プロンプトをベクトル化
user_prompt = input("プロンプトを入力してください: ")
vectorized_user_prompt = generate_embeddings(user_prompt)
print(f"ベクトル化したプロンプト{vectorized_prompt[:50]}")

# ベクトルクエリの作成
vector_query = VectorizedQuery(
    vector=vectorized_user_prompt,
    k_nearest_neighbors=3,  # 上位3件の結果を取得します
    fields="contentVector"  # ベクトル検索を行うフィールドを指定します
)

# ベクトル検索の実行
results = searchClient.search(
    search_text='',  # ベクトル検索のみ行うためテキストクエリは空
    vector_queries=[vector_query],
    select=['id', 'content'],
)

# 最初の検索結果を取得
first_result = next(results, None)
if not first_result:
    print("検索結果が見つかりませんでした")

# システムメッセージの定義
system_message_chat_conversation= """
あなたはユーザーの質問に回答するチャットボットです。
回答については、「Sources:」以下に記載されている内容に基づいて回答してください。
回答は簡潔にしてください。
「Sources:」に記載されている情報以外の回答はしないでください。
また、ユーザーの質問に対して、Sources:以下に記載されている内容に基づいて適切な回答ができない場合は、「すみません。わかりません。」と回答してください。
回答の中に情報源の提示は含めないでください。例えば、回答の中に「Sources:」という形で情報源を示すことはしないでください。
"""

# ユーザーメッセージの作成
user_message = """
{query}

Sources:
{source}
""".format(query=user_prompt, source=first_result["content"])

# メッセージリストの作成
messages_for_vector_answer = [
    {"role": "system", "content": system_message_chat_conversation},
    {"role": "user", "content": user_message}
]

# Azure OpenAI Serviceに回答生成を依頼
response = openAIClient.chat.completions.create(
    model="gpt-4o-mini-deploy",
    messages=messages_for_vector_answer
)

# 生成された回答を表示
response_text = response.choices[0].message.content
print(response_text)

ベクトル化したプロンプト[-0.007567174732685089, 0.01277646142989397, 0.07194299250841141, -0.009398650377988815, 0.03917383775115013, 0.012919032014906406, 0.012765495106577873, -0.005538294557482004, 0.021451294422149658, -0.01731676608324051, 0.0035368315875530243, 0.050316229462623596, -0.009097060188651085, 0.009820876643061638, -0.010478892363607883, -0.03226467967033386, -0.019872058182954788, -0.03000549226999283, 0.021506130695343018, 0.04116981849074364, 0.011953942477703094, 0.024368496611714363, -0.024851040914654732, 0.03366844356060028, 0.005922136828303337, -0.013522212393581867, 0.035159945487976074, -0.002153628971427679, 0.024280760437250137, -0.05062330141663551, -0.0041701714508235455, -0.04320966452360153, -0.005724732298403978, -0.008680317550897598, -0.005409433040767908, 0.03362457826733589, 0.014662771485745907, 0.026627682149410248, -0.00857613142579794, -0.0404021330177784, -0.014070558361709118, -0.020617809146642685, 0.040116991847753525, -0.07957597076892853, -0.043275

<a id="extra2"></a>
## チャンクサイズによる回答内容の変化

ハンズオンの中では、チャンクサイズを1000としてインデクシングをしました。ここでは、チャンクサイズを3000とした際のRAGの回答内容の変化を見てみましょう。流れとしては、以下のとおりです。

1. チャンクサイズ3000のドキュメントをインデクシングするためのインデックス「docs2」を作成する
2. チャンクサイズ3000,オーバーラップ600でドキュメントをチャンク化する（チャンクサイズが小さいので終了までに）
3. チャンク化したドキュメントをdocs2にインデクシングする
4. 任意のプロンプトで回答を依頼する


In [11]:
# インデックス「docs2」を作成する
def create_index(index_name):
    """
    Azure AI Searchのインデックスを作成する
    """
    client = SearchIndexClient(endpoint=search_endpoint, credential=AzureKeyCredential(search_api_key))

    # すでにインデックスが作成済みである場合には何もしない
    if index_name in client.list_index_names():
        print(f"インデックス '{index_name}' はすでに作成済みです")
        return

    # インデックスのフィールドを定義する
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="content", type="Edm.String", analyzer_name="ja.microsoft"),
        SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                    searchable=True, vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile")
    ]

    # ベクトル検索のための定義を行う
    vector_search = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(
                name="myHnsw"
            )
        ],
        profiles=[
            VectorSearchProfile(
                name="myHnswProfile",
                algorithm_configuration_name="myHnsw",
            )
        ]
    )

    # インデックスを作成する
    index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search)
    client.create_index(index)
    print(f"インデックス '{index_name}' が作成されました")

# インデックス名を指定してインデックスを作成する
index_name = "docs2"
create_index(index_name)

インデックス 'docs2' が作成されました


In [12]:
def create_chunk(content: str, separator: str, chunk_size: int = 512, overlap: int = 0):
    """
    テキストを指定したサイズで分割する
    """
    splitter = RecursiveCharacterTextSplitter(chunk_overlap=overlap, chunk_size=chunk_size, separators=separator)
    chunks = splitter.split_text(content)
    return chunks

# テキストを指定したサイズで分割する
chunksize = 3000  # チャンクサイズ
overlap = 600  # オーバーラップサイズ
separator = ["\n\n", "\n", "。", "、", " ", ""]  # 区切り文字
chunks2 = create_chunk(content, separator, chunksize, overlap)
print("チャンク化完了")

チャンク化完了


In [13]:
# テキストをAzure AI Searchのdocs2にインデックスする
def index_docs(chunks: list):
    """
    ドキュメントをAzure AI Searchにインデックスする
    """
    # Azure AI SearchのAPIに接続するためのクライアントを生成する
    searchClient = SearchClient(
        endpoint=search_endpoint,
        index_name="docs2",
        credential=AzureKeyCredential(search_api_key)
    )

    # Azure OpenAIのAPIに接続するためのクライアントを生成する
    openAIClient = AzureOpenAI(
        api_key=aoai_api_key,
        api_version=aoai_api_version,
        azure_endpoint=aoai_endpoint
    )

    # チャンク化されたテキストとそのテキストのベクトルをAzure AI Searchにアップロードする
    for i, chunk in enumerate(chunks):
        print(f"{i+1}個目のチャンクを処理中...")
        response = openAIClient.embeddings.create(
            input=chunk,
            model="text-embedding-3-small-deploy"
        )

        # チャンク化されたテキストとそのテキストのベクトルをAzure AI Searchにアップロードする
        document = {"id": str(i), "content": chunk, "contentVector": response.data[0].embedding}
        searchClient.upload_documents([document])
    print("インデクシング完了")

# テキストをAzure AI Searchにインデックスする
index_docs(chunks2)

1個目のチャンクを処理中...
2個目のチャンクを処理中...
3個目のチャンクを処理中...
4個目のチャンクを処理中...
5個目のチャンクを処理中...
6個目のチャンクを処理中...
7個目のチャンクを処理中...
8個目のチャンクを処理中...
9個目のチャンクを処理中...
10個目のチャンクを処理中...
11個目のチャンクを処理中...
12個目のチャンクを処理中...
13個目のチャンクを処理中...
14個目のチャンクを処理中...
15個目のチャンクを処理中...
16個目のチャンクを処理中...
17個目のチャンクを処理中...
18個目のチャンクを処理中...
19個目のチャンクを処理中...
20個目のチャンクを処理中...
21個目のチャンクを処理中...
22個目のチャンクを処理中...
23個目のチャンクを処理中...
24個目のチャンクを処理中...
25個目のチャンクを処理中...
26個目のチャンクを処理中...
27個目のチャンクを処理中...
28個目のチャンクを処理中...
29個目のチャンクを処理中...
30個目のチャンクを処理中...
31個目のチャンクを処理中...
32個目のチャンクを処理中...
33個目のチャンクを処理中...
インデクシング完了


In [ ]:
# プロンプトをベクトル化
user_prompt = input("プロンプトを入力してください: ")
vectorized_user_prompt = generate_embeddings(user_prompt)
print(f"ベクトル化したプロンプト{vectorized_prompt[:50]}")

searchClient = SearchClient(
    endpoint=search_endpoint,
    index_name="docs2",
    credential=AzureKeyCredential(search_api_key)
)

# ベクトルクエリの作成
vector_query = VectorizedQuery(
    vector=vectorized_user_prompt,
    k_nearest_neighbors=3,  # 上位3件の結果を取得します
    fields="contentVector"  # ベクトル検索を行うフィールドを指定します
)

# ベクトル検索の実行
results = searchClient.search(
    search_text='',  # ベクトル検索のみ行うためテキストクエリは空
    vector_queries=[vector_query],
    select=['id', 'content'],
)

# 最初の検索結果を取得
first_result = next(results, None)
if not first_result:
    print("検索結果が見つかりませんでした")

# システムメッセージの定義
system_message_chat_conversation= """
あなたはユーザーの質問に回答するチャットボットです。
回答については、「Sources:」以下に記載されている内容に基づいて回答してください。
回答は簡潔にしてください。
「Sources:」に記載されている情報以外の回答はしないでください。
また、ユーザーの質問に対して、Sources:以下に記載されている内容に基づいて適切な回答ができない場合は、「すみません。わかりません。」と回答してください。
回答の中に情報源の提示は含めないでください。例えば、回答の中に「Sources:」という形で情報源を示すことはしないでください。
"""

# ユーザーメッセージの作成
user_message = """
{query}

Sources:
{source}
""".format(query=user_prompt, source=first_result["content"])

# メッセージリストの作成
messages_for_vector_answer = [
    {"role": "system", "content": system_message_chat_conversation},
    {"role": "user", "content": user_message}
]

# Azure OpenAI Serviceに回答生成を依頼
response = openAIClient.chat.completions.create(
    model="gpt-4o-mini-deploy",
    messages=messages_for_vector_answer
)

# 生成された回答を表示
response_text = response.choices[0].message.content
print(response_text)

## 追加コンテンツは以上になります
不要なコストの発生を避けるために[Azureリソースの削除・Codespacesの停止](#delete)をお願い申し上げます。